# LoRA 간단구현

Low-Rank Adaptation

<img src='https://d.pr/i/kWZ843+' width=500/>

대형 언어 모델(LLM)이나 Transformer 계열에서 **전체 파라미터를 직접 미세조정(fine-tuning)하지 않고**, 주요 가중치(주로 Linear 레이어)에 **저차원 행렬(저랭크 행렬)** 로 변경만을 학습하는 방법이다.


**LoRA 공식**

1. 기존 Linear 연산: $Y = W X$
2. LoRA 적용: $Y = (W + \Delta W) X$
  - $\Delta W = B A$
  - 여기서 $r \ll d, k$
3. 즉, $\Delta W$는 **저랭크 행렬 곱**으로 근사화한다.



**원리:**
* 기존의 큰 가중치 행렬 $W$ 대신, $W + BA$ 형태로 쓴다.
* 여기서 $B \in \mathbb{R}^{d \times r}$, $A \in \mathbb{R}^{r \times k}$는 **r이 작은 값(저랭크)** $B, A$만 학습, $W$는 동결(freeze)한다.

**설명:**
* $d$: 원래 가중치 $W\in\mathbb R^{d\times k}$의 **행(row) 크기**, 곧 **입력(input) 차원**이다.
* $k$: $W$의 **열(column) 크기**, 곧 **출력(output) 차원**이다.
* $r$: 어댑터 $A\in\mathbb R^{r\times k}, B\in\mathbb R^{d\times r}$의 **내적(rank) 차원**으로, $r\ll\min(d,k)$인 작은 값이다.

* 대문자 ℝ는 “Real numbers”, 곧 **실수 전체의 집합**을 뜻한다.
* ℝᵈ는 d차원 실수 벡터 공간
* ℝ^{d×k}는 d×k 크기의 실수 행렬들이 모인 공간을 의미한다.

* 저차원 행렬 초기화
  * A ∈ ℝ^{r×d}의 각 원소는 평균 0, 분산 σ²를 갖는 정규분포에서 샘플링해 초기화한다.
  * B ∈ ℝ^{d×r}는 전부 0으로 초기화한다.
  * 이렇게 하면 초반에는 B·A = 0 이므로 원래의 W만 동작하고, 학습이 진행되면서 B만 업데이트해 가벼운 파라미터 조정이 가능하다.


**장점:**

* 파라미터 수와 GPU 메모리 사용량 대폭 감소
* 기존 Pretrained Model의 성능을 최대한 보존
* 다양한 작업에 손쉽게 맞춤화(Prompt Tuning, Adapter 등과 유사한 "파인튜닝 경량화" 전략)



* [LoRA 논문 - LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
* 실제 적용은 Hugging Face `peft` 라이브러리에서 훨씬 더 편하게 쓸 수 있다.


### LoRA 간단구현

In [1]:
import torch

# 입력값 X
X = torch.randn(3, 8) # (batch, in_feature)

# 기존 모델의 가중치 (in_feature, out_feature): (8, 4)
W = torch.randn(4, 8)

# 저차원행렬 A, B
# A: (r, in_feature)
# B: (out_feature, r)
r = 2
A = torch.randn(r, 8)
B = torch.randn(4, r)

# LoRA 적용
delta_W = B @ A

Y_lora = X @ (W + delta_W).T

print('LoRA적용: ', Y_lora.shape) # (batch, out_feature)

LoRA적용:  torch.Size([3, 4])


In [2]:
# LoRA 클래스 예제: alpha
import math
import torch.nn as nn

class LoRALinear(nn.Module):
    def __init__(self, in_feature, out_feature, r=2, alpha=2):
        super().__init__()
        self.r = r
        self.alpha = alpha
        self.W = nn.Linear(in_feature, out_feature, bias=False)

        # 저차원 어탭터행렬
        self.A = nn.Parameter(torch.zeros((r, in_feature)))
        self.B = nn.Parameter(torch.zeros((out_feature, r)))

        # 초기화(논문제안)
        nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))
        nn.init.zeros_(self.B)    

    def forward(self, x):
        W_x = self.W(x)
        lora = (self.B @ self.A) @ x.T
        lora = lora.T * self.alpha / self.r # alpha와 r의 비율에 따라 lora의 적용정도 제한
        return W_x + lora
    
lora_layer = LoRALinear(8, 4)
X = torch.randn(3, 8)
Y = lora_layer(X)
print(Y.shape)
print(Y)

torch.Size([3, 4])
tensor([[ 0.4208,  0.5394, -0.0283, -0.3039],
        [-1.1607, -0.6206,  0.3516,  0.5049],
        [ 0.6086,  0.3651,  0.0542,  1.0664]], grad_fn=<AddBackward0>)
